#### Let's run this notebook from our GitHub repo! ####

In [1]:
using System;
using System.Collections.Generic;

Dictionary<string, string> dictionary= new Dictionary<string, string>();
dictionary.Add("alice", "Learning Azure Cosmos DB");
dictionary.Add("bob", "Running a query");
dictionary.Add("carol", "Using a notebook");

dictionary

System.Collections.Generic.Dictionary`2[System.String,System.String]